In [1]:
import torch
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import torch
import cv2
import mediapipe as mp
import requests
from torchvision import transforms
from PIL import Image
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
words_corpus = pd.read_csv(r"words_data.csv")
word_frames = pd.read_excel("ISL_CSLRT_Corpus_word_details.xlsx")

In [3]:
words_corpus.head()

,Unnamed: 0,Word,0,1,2,3,4,5,6,7,...,54,55,56,57,58,59,60,61,62,63
0,0,A LOT,0.316836,0.439745,7.139941e-08,0.322064,0.408899,-0.012561,0.320306,0.373865,...,0.260467,0.404656,-0.028454,0.251036,0.396338,-0.031925,0.241957,0.388138,-0.034238,0.061003
1,1,A LOT,0.777162,0.594822,1.888133e-07,0.766787,0.569503,-0.029101,0.766939,0.543866,...,0.831578,0.531044,-0.019791,0.839598,0.514735,-0.025115,0.845963,0.498970,-0.028685,0.066329
2,2,A LOT,0.282781,0.426264,9.168880e-08,0.279939,0.391712,0.005966,0.275245,0.354410,...,0.289899,0.329482,-0.034678,0.295946,0.304782,-0.038501,0.301991,0.284427,-0.040737,0.070718
3,3,A LOT,0.613345,0.728356,7.959181e-08,0.623554,0.698023,-0.007444,0.642625,0.672808,...,0.660544,0.768533,-0.024703,0.665910,0.764937,-0.027244,0.669510,0.756921,-0.028706,0.074697
4,4,ABUSE,0.493275,0.434260,8.329591e-08,0.508639,0.422977,-0.002672,0.521094,0.407595,...,0.498473,0.365121,-0.024746,0.509517,0.378327,-0.025282,0.514609,0.389408,-0.024399,0.016817


In [4]:
words_corpus.shape

(984, 66)

In [5]:
word_frames.head()

,Word,Frames path
0,A LOT,ISL_CSLRT_Corpus\Frames_Word_Level\A LOT\A LOT...
1,A LOT,ISL_CSLRT_Corpus\Frames_Word_Level\A LOT\A LOT...
2,A LOT,ISL_CSLRT_Corpus\Frames_Word_Level\A LOT\A LOT...
3,A LOT,ISL_CSLRT_Corpus\Frames_Word_Level\A LOT\A LOT...
4,ABUSE,ISL_CSLRT_Corpus\Frames_Word_Level\ABUSE\ABUSE...


In [6]:
word_frames['Frames path'] = word_frames['Frames path'].str.replace(r'^ISL_CSLRT_Corpus', '', regex=True)

# frame_path = word_frames.iloc[2][1]

file_url = r"C:\Users\hp\Indian-Sign-Language-SIH\dataset"

word_frames['Frames path'] = file_url + word_frames['Frames path']


In [7]:
word_frames.head()

,Word,Frames path
0,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
1,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
2,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
4,ABUSE,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...


In [8]:
word_frames.iloc[2]["Frames path"]

'C:\\Users\\hp\\Indian-Sign-Language-SIH\\dataset\\Frames_Word_Level\\A LOT\\A LOT (3).jpg'

In [9]:
word_frames.shape

(1036, 2)

In [10]:
words_corpus = words_corpus[["Unnamed: 0", "Word"]]

In [11]:
words_corpus

,Unnamed: 0,Word
0,0,A LOT
1,1,A LOT
2,2,A LOT
3,3,A LOT
4,4,ABUSE
...,...,...
979,1031,YOU
980,1032,YOU
981,1033,YOU
982,1034,YOU


In [12]:
words_corpus.shape

(984, 2)

In [13]:
words_corpus["Word"].value_counts()

Word
YOU             105
I_ME_MINE_MY     92
WHAT             28
HELP             22
FOOD             22
               ... 
ALL               3
THINGS            3
DON'T CARE        2
BEAUTIFUL         2
PLEASE            2
Name: count, Length: 114, dtype: int64

In [14]:
filtered_words_df = words_corpus[words_corpus["Word"].map(words_corpus["Word"].value_counts()) <= 28]

In [15]:
filtered_words = filtered_words_df[["Unnamed: 0", "Word"]]
filtered_words

,Unnamed: 0,Word
0,0,A LOT
1,1,A LOT
2,2,A LOT
3,3,A LOT
4,4,ABUSE
...,...,...
874,920,WHO
875,922,WORRY
876,923,WORRY
877,924,WORRY


In [16]:
filtered_words["Word"].unique()

array(['A LOT', 'ABUSE', 'AFRAID', 'AGREE', 'ALL', 'ANGRY', 'ANYTHING',
       'APPRECIATE', 'BAD', 'BEAUTIFUL', 'BECOME', 'BED', 'BORED',
       'BRING', 'CHAT', 'CLASS', 'COLD', 'COLLEGE_SCHOOL', 'COMB', 'COME',
       'CONGRATULATIONS', 'CRYING', 'DARE', 'DIFFERENCE', 'DILEMMA',
       'DISAPPOINTED', 'DO', "DON'T CARE", 'ENJOY', 'FAVOUR', 'FEVER',
       'FINE', 'FOOD', 'FREE', 'FRIEND', 'FROM', 'GO', 'GOOD', 'GRATEFUL',
       'HAD', 'HAPPENED', 'HAPPY', 'HEAR', 'HEART', 'HELLO_HI', 'HELP',
       'HIDING', 'HOW', 'HUNGRY', 'HURT', 'KIND', 'LEAVE', 'LIKE',
       'LIKE_LOVE', 'MEAN IT', 'MEDICINE', 'MEET', 'NAME', 'NICE', 'NOT',
       'NUMBER', 'OLD_AGE', 'ON THE WAY', 'OUTSIDE', 'PHONE', 'PLACE',
       'PLEASE', 'POUR', 'PREPARE', 'PROMISE', 'REALLY', 'REPEAT', 'ROOM',
       'SERVE', 'SHIRT', 'SITTING', 'SLEEP', 'SLOWER', 'SO MUCH',
       'SOFTLY', 'SOME HOW', 'SOME ONE', 'SOMETHING', 'SORRY', 'SPEAK',
       'STOP', 'STUBBORN', 'SURE', 'TAKE CARE', 'TAKE TIME', 'TALK',
     

In [17]:
filtered_words["Word"].value_counts()

Word
WHAT          28
FOOD          22
HELP          22
WATER         20
HOW           19
              ..
MEAN IT        3
ALL            3
PLEASE         2
DON'T CARE     2
BEAUTIFUL      2
Name: count, Length: 112, dtype: int64

In [18]:
filtered_words = filtered_words.set_index("Unnamed: 0")
filtered_words.tail(2)

,Word
Unnamed: 0,
924,WORRY
925,WORRY


In [19]:
filtered_words.drop_duplicates(inplace=True)

In [20]:
filtered_words.shape

(112, 1)

In [21]:
filtered_rows_1 = filtered_words[["Word"]].join(word_frames["Frames path"], on=filtered_words.index)

In [22]:
filtered_rows_1

,Word,Frames path
Unnamed: 0,,
0,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
4,ABUSE,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
9,AFRAID,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
15,AGREE,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
22,ALL,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
...,...,...
872,WELCOME,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
877,WHAT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
908,WHERE,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...


In [23]:
filtered_rows_1.iloc[111, 1]

'C:\\Users\\hp\\Indian-Sign-Language-SIH\\dataset\\Frames_Word_Level\\WORRY\\WORRY (1).jpg'

In [24]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(20, shear=10),
    transforms.ToTensor(),
    transforms.Resize((1080, 1920))  
])

# Convert tensor back to PIL image
to_pil = transforms.ToPILImage()

In [25]:
augmented_data = []

for _, row in filtered_rows_1.iterrows():
    word = row['Word']
    image_path = row['Frames path']
    
    print(f"Opening image at path: {image_path}")
    
    try:
        # Open the image
        image = Image.open(image_path)
        
        # Apply transformations 28 times
        for i in range(28):
            augmented_tensor = transform(image)  # Get the transformed tensor
            augmented_image = to_pil(augmented_tensor)  # Convert to PIL image
            
            # Create the augmented image path
            augmented_image_path = image_path.replace('.jpg', f'_aug_{i}.jpg')
            
            # Save the augmented image to the same directory
            augmented_image.save(augmented_image_path)
            
            # Append details to the augmented data list
            augmented_data.append({'Word': word, 'Frames path': augmented_image_path})
    
    except FileNotFoundError as e:
        print(f"FileNotFoundError: {e}")

# Create a new DataFrame with the augmented data
augmented_df = pd.DataFrame(augmented_data)

# Optionally, you can concatenate the original dataframe with the augmented one
# final_df = pd.concat([df, augmented_df], ignore_index=True)

Opening image at path: C:\Users\hp\Indian-Sign-Language-SIH\dataset\Frames_Word_Level\A LOT\A LOT (1).jpg
Opening image at path: C:\Users\hp\Indian-Sign-Language-SIH\dataset\Frames_Word_Level\ABUSE\ABUSE (1).jpg
Opening image at path: C:\Users\hp\Indian-Sign-Language-SIH\dataset\Frames_Word_Level\AFRAID\AFRAID (1).jpg
Opening image at path: C:\Users\hp\Indian-Sign-Language-SIH\dataset\Frames_Word_Level\AGREE\AGREE (1).jpg
Opening image at path: C:\Users\hp\Indian-Sign-Language-SIH\dataset\Frames_Word_Level\ALL\ALL (1).jpg
Opening image at path: C:\Users\hp\Indian-Sign-Language-SIH\dataset\Frames_Word_Level\ANGRY\ANGRY (1).jpg
Opening image at path: C:\Users\hp\Indian-Sign-Language-SIH\dataset\Frames_Word_Level\ANYTHING\ANYTHING (1).jpg
Opening image at path: C:\Users\hp\Indian-Sign-Language-SIH\dataset\Frames_Word_Level\APPRECIATE\APPRECIATE (1).jpg
Opening image at path: C:\Users\hp\Indian-Sign-Language-SIH\dataset\Frames_Word_Level\BAD\BAD (1).jpg
Opening image at path: C:\Users\hp\I

In [26]:
augmented_df.shape

(3136, 2)

In [27]:
augmented_df.head()

,Word,Frames path
0,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
1,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
2,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
4,A LOT,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...


In [28]:
# # Set the minimum number of data points per word
# min_data_points = 2

# # Group by 'Word' and apply sampling to ensure each word has only `min_data_points` rows
# consistent_word_frames = word_frames.groupby('Word').apply(lambda x: x.sample(n=min_data_points, random_state=369) if len(x) >= min_data_points else x).reset_index(drop=True)


In [29]:
# consistent_word_frames.shape

In [30]:
## the data points which were not augmented need to be included in dataset

filtered_words_df2 = words_corpus[words_corpus["Word"].map(words_corpus["Word"].value_counts()) > 28]

filtered_words_df2

,Unnamed: 0,Word
357,377,I_ME_MINE_MY
358,378,I_ME_MINE_MY
359,379,I_ME_MINE_MY
360,380,I_ME_MINE_MY
361,381,I_ME_MINE_MY
...,...,...
979,1031,YOU
980,1032,YOU
981,1033,YOU
982,1034,YOU


In [31]:
filtered_words_df2 = filtered_words_df2[["Unnamed: 0", "Word"]]

filtered_words_df2 = filtered_words_df2.set_index("Unnamed: 0")

filtered_rows_2 = filtered_words_df2[["Word"]].join(word_frames["Frames path"], on=filtered_words_df2.index)

In [32]:
filtered_rows_2.shape

(197, 2)

In [33]:
filtered_rows_2

,Word,Frames path
Unnamed: 0,,
377,I_ME_MINE_MY,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
378,I_ME_MINE_MY,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
379,I_ME_MINE_MY,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
380,I_ME_MINE_MY,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
381,I_ME_MINE_MY,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
...,...,...
1031,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
1032,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
1033,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...


In [34]:
I_corpus = filtered_rows_2[:28]

You_corpus = filtered_rows_2[len(filtered_rows_2)-28:]

In [35]:
I_corpus.shape

(28, 2)

In [36]:
You_corpus.shape

(28, 2)

In [37]:
df = pd.concat([augmented_df, I_corpus, You_corpus], ignore_index=True)

In [38]:
df.tail()

,Word,Frames path
3187,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3188,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3189,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3190,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...
3191,YOU,C:\Users\hp\Indian-Sign-Language-SIH\dataset\F...


In [39]:
df.shape

(3192, 2)

In [40]:
df.to_csv("augmented_df.csv")